# $group 的基本操作

In [ ]:
# grouping by year and getting a count per year using the { $sum: 1 } pattern
db.movies.aggregate([
    {
        $group: {
            _id: "$year",
            "numFilmsThisYear": { $sum: 1 }
        }
    }
]).pretty()

In [ ]:
# grouping as before, then sorting in descending order based on the count
db.movies.aggregate([
    {
        $group: {
            _id: "$year",
            "count": { $sum: 1 }
        }
    },
    { $sort: { count: -1 }}
]).pretty()

In [ ]:
# grouping on the number of directors a film has, demonstrating that we have to
# validate types to protect some expressions
db.movies.aggregate([
    {
        $group: {
            _id: { "numDirectors": { $cond: [ { $isArray: "$directors" }, { $size: "$directors" }, 0 ] } },
            numFilms: { $sum: 1},
            averageMetacritic: { $avg: "$metacritic" }
        }
    },
    { $sort: { "_id.numDirectors": -1 }}
]).pretty()

In [ ]:
# showing how to group all documents together. By convention, we use null or an empty string, ""
db.movies.aggregate([
    {
        $group: {
            _id: null,
            count: { $sum: 1 }
        }
    }
])

In [ ]:
# filtering results to only get documents with a numeric metacritic value
db.movies.aggregate([
    {
        $match: {
            metacritic: { $gte: 0 }
        }
    },
    {
        $group: {
            _id: null,
            averageMetacritic: { $avg: "$metacritic" }
        }
    }
]).pretty()

# Accumulator Expressions 的基本操作

In [ ]:
# using $reduce to get the highest temperature
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            max_high: {
                $reduce: {
                    input: "$trends",
                    initialValue: -Infinity,
                    in: { $cond: [ { $gt: [ "$$this.avg_high_tmp", "$$value" ]}, "$$this.avg_high_tmp", "$$value" ]}
                }
            }
        }
    }
])

In [ ]:
# performing the inverse, grabbing the lowest temperature
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            min_high: {
                $reduce: {
                    input: "$trends",
                    initialValue: Infinity,
                    in: { $cond: [ { $lt: [ "$$this.avg_low_tmp", "$$value" ]}, "$$this.avg_low_tmp", "$$value" ]}
                }
            }
        }
    }
])

In [ ]:
# note that these two operations can be done with the following operations can
# be done more simply. The following two expressions are functionally identical
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            max_high: { $max: "$trends.avg_high_tmp" }
        }
    }
])

db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            min_high: { $min: "$trends.avg_low_tmp" }
        }
    }
])

In [ ]:
# getting the average and standard deviations of the consumer price index
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            average_cpi: { $avg: "$trends.icecream_cpi"},
            cpi_deviation: { $stdDevPop: "$trends.icecream_cpi" }
        }
    }
])

In [ ]:
# using the $sum expression to get total yearly sales
db.icecream_data.aggregate([
    {
      $project: {
        _id: 0,
        "yearly_sales (millions)": { $sum: "$trends.icecream_sales_in_millions" }
      }
    }
  ])

### 範例01

In [ ]:
db.movies.aggregate([
    {
        $match: {
            "imdb.votes": { $gte: 1}
        }
    },
    {
        $group: {
            _id: null,
            maxVotes: { $max: "$imdb.votes" },
            minVotes: { $min: "$imdb.votes" }
        }
    }
])

### 範例02

In [ ]:
db.grades.aggregate([
    {
        $project: {
            maxGrade: { $max: "$quizzes" }
        }
    }
])

### 範例03

In [ ]:
# 參考解答
db.movies.aggregate([
    {
        $match: { 
            awards: { $exists: true }
        }
    },
    {
        $project: {
            _id: 0,
            title: 1,
            awards: { $split: [ "$awards", " " ]},
            imdb_rating: "$imdb.rating" 
        }
    },
    {
        $match: {

            "awards.0": "Won",
            "awards.2": { $in: [ "Oscar.", "Oscars." ]}
        }
    },
    {
        $group: {
            _id: null,
            highest_rating: { $max: "$imdb_rating" },
            lowest_rating: { $min: "$imdb_rating" },
            average_rating: { $avg: "$imdb_rating" },
            deviation: { $stdDevSamp: "$imdb_rating" }
        }
    }
]).pretty()

In [ ]:
# 參考解答
db.movies.aggregate([
  {
    $match: {
      awards: /Won \d{1,2} Oscars?/
    }
  },
  {
    $group: {
      _id: null,
      highest_rating: { $max: "$imdb.rating" },
      lowest_rating: { $min: "$imdb.rating" },
      average_rating: { $avg: "$imdb.rating" },
      deviation: { $stdDevSamp: "$imdb.rating" }
    }
  }
])pretty()

### 範例04-有時不用 $group 更能增加效能

In [ ]:
# working within the arrays is always better if we want to do analysis within
# a document. We get the same results in a slighlty easier to work with format
# and didn't incur the cost of a $group stage
db.stocks.aggregate([
    {
        $project: {
            buy_actions: {
                $size: {
                    $filter: {
                        input: "$trades",
                        cond: { $eq: [ "$$this.action", "buy" ]}
                    }
                }
            },
            sell_actions: {
                $size: {
                    $filter: {
                        input: "$trades",
                        cond: { $eq: [ "$$this.action", "sell" ]}
                    }                       
                }
            },
            total_trades: { $size: "$trades" }
        }
    },
    {
        $sort: { total_trades: -1 }
    }
]).pretty()

### 範例05-有時不用 $group 更能增加效能

In [ ]:
# remember, expression composition is powerful. Be creative, and things
# that can be done inline. Notice that there is no intermediary stage to
# filter the trades array first, it's just done as part of the argument to
# the reduce expression.
db.stocks.aggregate([
    {
        $project: {
            _id: 0,
            mdb_only: {
                $reduce: {
                    input: {
                        $filter: {
                            input: "$trades",
                            cond: { "eq": [ "$$this.ticker", "MDB" ]}
                        }
                    },
                    initialValue: {
                        buy: { total_count: 0, total_value: 0},
                        sell: { total_count: 0, total_value: 0}
                    },
                    in: {
                        $cond: [
                            { $eq: [ "$$this.action", "buy" ]},
                            {
                                buy: {
                                    total_count: { $add: [ "$$value.buy.total_count", 1 ]},
                                    total_value: { $add: [ "$$value.buy.total_value", "$$this.price"]}
                                },
                                sell: "$$value.sell"
                            },
                            {
                                sell: {
                                    total_count: { $add: [ "$$value.sell.total_count", 1 ]},
                                    total_value: { $add: [ "$$value.sell.total_value", "$$this.price"]}
                                },
                                buy: "$$value.buy"
                            }
                        ]
                    }
                }
            }     
        }
    }
]).pretty()